# Incorporating Domain Knowledge

Aside from tuning the solver parameters (c, k, alpha), MCTS currently offers several means of incorporating domain knowledge. The following solver parameters control the planner's behavior:

- `estimate_value` determines how the value is estimated at the leaf nodes (this is usually done using a rollout simulation).
- `init_N` and `init_Q` determine how N(s,a) and Q(s,a) are initialized when a new node is created.
- `next_action` determines which new actions are tried in double progressive widening

There are three ways of specifying these parameters: 1) with constant values, 2) with functions, and 3) with custom objects.

In [1]:
using MCTS
importall POMDPs
using POMDPModels
mdp = GridWorld();

## Constant Values

`init_N`, `init_Q`, and `estimate_value` can be set with constant values (though this is a bad idea for `estimate_value`. `next_action` cannot be specified in this way. The following code sets all new N to 3 and all new Q to 11.73 for example.

In [2]:
solver = MCTSSolver(n_iterations=3, depth=4,
                    init_N=3,
                    init_Q=11.73)
policy = solve(solver, mdp)
action(policy, GridWorldState(1,1))
println("State-Action Nodes")
for (s,sn) in policy.tree
    for san in sn.sanodes
        println("s:$s, a:$(san.action) Q:$(san.Q) N:$(san.N)")
    end
end

State-Action Nodes
s:POMDPModels.GridWorldState(1, 1, false), a:up Q:8.7975 N:4
s:POMDPModels.GridWorldState(1, 1, false), a:down Q:5.027142857142857 N:7
s:POMDPModels.GridWorldState(1, 1, false), a:left Q:11.73 N:3
s:POMDPModels.GridWorldState(1, 1, false), a:right Q:11.73 N:3
s:POMDPModels.GridWorldState(1, 2, false), a:up Q:11.73 N:3
s:POMDPModels.GridWorldState(1, 2, false), a:down Q:11.73 N:3
s:POMDPModels.GridWorldState(1, 2, false), a:left Q:11.73 N:3
s:POMDPModels.GridWorldState(1, 2, false), a:right Q:11.73 N:3


## Functions

`init_N`, `init_Q`, `estimate_value`, and `next_action` can also be functions. The following code will

- initialize Q to 0.0 everywhere except state [1,2] where it will be 11.73
- initialize N to 0 everywhere except state [1,2] where it will be 3
- estimate the value to be 10 divided by the manhattan distance to state [9,3]
- always choose action "up" first in double progressive widening

Note: the `?` below is part of the [ternary operator](http://docs.julialang.org/en/release-0.5/manual/control-flow/#control-flow).

In [3]:
special_Q(mdp, s, a) = s == GridWorldState(1,2) ? 11.73 : 0.0
special_N(mdp, s, a) = s == GridWorldState(1,2) ? 3 : 0

function manhattan_value(mdp, s, depth) # depth is the solver `depth` parameter less the number of timesteps that have already passed (it can be ignored in many cases)
    m_dist = abs(s.x-9)+abs(s.y-3)
    val = 10.0/m_dist
    println("Set value for $s to $val") # this is not necessary - just shows that it's working later
    return val
end

function up_priority(mdp, s, snode) # snode is the state node of type DPWStateNode
    if haskey(snode.A, GridWorldAction(:up)) # "up" is already there
        return GridWorldAction(rand([:up, :left, :down, :right])) # add a random action
    else
        return GridWorldAction(:up)
    end
end;

In [4]:
solver = DPWSolver(n_iterations=8, depth=4,
                   init_N=special_N, init_Q=special_Q,
                   estimate_value=manhattan_value,
                   next_action=up_priority)
policy = solve(solver, mdp)
action(policy, GridWorldState(1,1))
println("State-Action Nodes:")
for (s,sn) in policy.tree
    for (a,san) in sn.A
        println("s:$s, a:$a Q:$(san.Q) N:$(san.N)")
    end
end

Set value for POMDPModels.GridWorldState(1, 1, false) to 1.0
Set value for POMDPModels.GridWorldState(1, 2, false) to 1.1111111111111112
Set value for POMDPModels.GridWorldState(2, 1, false) to 1.1111111111111112
Set value for POMDPModels.GridWorldState(3, 1, false) to 1.25
Set value for POMDPModels.GridWorldState(1, 3, false) to 1.25
Set value for POMDPModels.GridWorldState(2, 2, false) to 1.25
State-Action Nodes:
s:POMDPModels.GridWorldState(1, 1, false), a:left Q:0.0 N:3
s:POMDPModels.GridWorldState(1, 1, false), a:right Q:1.1039351851851853 N:3
s:POMDPModels.GridWorldState(1, 1, false), a:up Q:0.7278935185185185 N:3
s:POMDPModels.GridWorldState(1, 1, false), a:down Q:0.0 N:1
s:POMDPModels.GridWorldState(3, 1, false), a:up Q:0.0 N:1
s:POMDPModels.GridWorldState(2, 1, false), a:right Q:0.0 N:1
s:POMDPModels.GridWorldState(2, 1, false), a:up Q:1.1875 N:2
s:POMDPModels.GridWorldState(2, 1, false), a:down Q:0.0 N:1
s:POMDPModels.GridWorldState(1, 2, false), a:up Q:9.094375 N:4


## Objects

There are many cases where functions are not suitable, for example when the solver needs to be serialized. In this case, arbitrary objects may be passed to the solver to encode the behavior. The same object can be passed to multiple solver parameters to govern all of their behavior. See the docstring for the solver for more information on which functions will be called on the object(s). The following code does exactly the same thing as the function-based code above:

In [5]:
mutable struct MyHeuristic
    target_state::GridWorldState
    special_state::GridWorldState
    special_Q::Float64
    special_N::Int
    priority_action::GridWorldAction
    rng::AbstractRNG
end;

In [6]:
MCTS.init_Q(h::MyHeuristic, mdp::GridWorld, s, a) = s == h.special_state ? h.special_Q : 0.0
MCTS.init_N(h::MyHeuristic, mdp::GridWorld, s, a) = s == h.special_state ? h.special_N : 0

function MCTS.estimate_value(h::MyHeuristic, mdp::GridWorld, s, depth::Int)
    targ = h.target_state
    m_dist = abs(s.x-targ.x)+abs(s.y-targ.y)
    val = 10.0/m_dist
    println("Set value for $s to $val") # this is not necessary - just shows that it's working later
    return val
end

function MCTS.next_action(h::MyHeuristic, mdp::GridWorld, s, snode::DPWStateNode)
    if haskey(snode.A, h.priority_action)
        return GridWorldAction(rand(h.rng, [:up, :left, :down, :right])) # add a random other action
    else
        return h.priority_action
    end
end;

In [7]:
heur = MyHeuristic(GridWorldState(9,3), GridWorldState(1,2), 11.73, 3, GridWorldAction(:up), Base.GLOBAL_RNG)
solver = DPWSolver(n_iterations=8, depth=4,
                   init_N=heur, init_Q=heur,
                   estimate_value=heur,
                   next_action=heur)
policy = solve(solver, mdp)
action(policy, GridWorldState(1,1))
println("State-Action Nodes:")
for (s,sn) in policy.tree
    for (a,san) in sn.A
        println("s:$s, a:$a Q:$(san.Q) N:$(san.N)")
    end
end

Set value for POMDPModels.GridWorldState(1, 1, false) to 1.0
Set value for POMDPModels.GridWorldState(1, 2, false) to 1.1111111111111112
Set value for POMDPModels.GridWorldState(2, 1, false) to 1.1111111111111112
Set value for POMDPModels.GridWorldState(2, 2, false) to 1.25
Set value for POMDPModels.GridWorldState(3, 1, false) to 1.25
State-Action Nodes:
s:POMDPModels.GridWorldState(1, 1, false), a:left Q:0.0 N:4
s:POMDPModels.GridWorldState(1, 1, false), a:right Q:0.5459201388888888 N:4
s:POMDPModels.GridWorldState(1, 1, false), a:up Q:1.063637152777778 N:2
s:POMDPModels.GridWorldState(1, 1, false), a:down Q:0.0 N:4
s:POMDPModels.GridWorldState(2, 1, false), a:left Q:0.0 N:1
s:POMDPModels.GridWorldState(2, 1, false), a:right Q:1.1875 N:1
s:POMDPModels.GridWorldState(2, 1, false), a:up Q:1.1875 N:1
s:POMDPModels.GridWorldState(2, 1, false), a:down Q:1.128125 N:1
s:POMDPModels.GridWorldState(1, 2, false), a:up Q:8.7975 N:4


## Rollouts

The most common way to estimate the value of a state node is with rollout simulations. This can be done with an arbitrary policy or solver by passing a `RolloutEstimator` object as the `estimate_value` parameter. The following code does this with a policy that moves towards state [9,3].

In [8]:
mutable struct SeekTarget <: Policy
    target::GridWorldState
end

In [9]:
function POMDPs.action(p::SeekTarget, s::GridWorldState, a::GridWorldAction=GridWorldAction(:up))
    if p.target.x > s.x
        return GridWorldAction(:right)
    elseif p.target.x < s.x
        return GridWorldAction(:left)
    elseif p.target.y > s.y
        return GridWorldAction(:up)
    else
        return GridWorldAction(:down)
    end
end

In [14]:
solver = MCTSSolver(n_iterations=5, depth=20,
                    estimate_value=RolloutEstimator(SeekTarget(GridWorldState(9,3))))
policy = solve(solver, mdp)
action(policy, GridWorldState(5,1))
println("State-Action Nodes")
for (s,sn) in policy.tree
    for san in sn.sanodes
        println("s:$s, a:$(san.action) Q:$(san.Q) N:$(san.N)")
    end
end

State-Action Nodes
s:POMDPModels.GridWorldState(4,2,false), a:POMDPModels.GridWorldAction(:up) Q:0.0 N:0
s:POMDPModels.GridWorldState(4,2,false), a:POMDPModels.GridWorldAction(:down) Q:0.0 N:0
s:POMDPModels.GridWorldState(4,2,false), a:POMDPModels.GridWorldAction(:left) Q:0.0 N:0
s:POMDPModels.GridWorldState(4,2,false), a:POMDPModels.GridWorldAction(:right) Q:0.0 N:0
s:POMDPModels.GridWorldState(4,1,false), a:POMDPModels.GridWorldAction(:up) Q:5.688000922764596 N:1
s:POMDPModels.GridWorldState(4,1,false), a:POMDPModels.GridWorldAction(:down) Q:0.0 N:0
s:POMDPModels.GridWorldState(4,1,false), a:POMDPModels.GridWorldAction(:left) Q:0.0 N:0
s:POMDPModels.GridWorldState(4,1,false), a:POMDPModels.GridWorldAction(:right) Q:0.0 N:0
s:POMDPModels.GridWorldState(5,1,false), a:POMDPModels.GridWorldAction(:up) Q:7.350918906249999 N:1
s:POMDPModels.GridWorldState(5,1,false), a:POMDPModels.GridWorldAction(:down) Q:0.0 N:20
s:POMDPModels.GridWorldState(5,1,false), a:POMDPModels.GridWorldAction(:left